In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

C:\Users\test.DESKTOP-ODO5P7A\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 1510 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:1510/status,
Dashboard: http://127.0.0.1:1510/status,Workers: 4
Total threads: 4,Total memory: 63.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:1511,Workers: 4
Dashboard: http://127.0.0.1:1510/status,Total threads: 4
Started: Just now,Total memory: 63.92 GiB
Comm: tcp://127.0.0.1:1526,Total threads: 1
Dashboard: http://127.0.0.1:1533/status,Memory: 15.98 GiB
Nanny: tcp://127.0.0.1:1514,


In [2]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk



RuntimeError: The mimesis module is required for this function.  Try:
  python -m pip install mimesis

In [3]:
!head -n 2 data/0.json

'head' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import dask.bag as db
import json
b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=40>

In [5]:
b.take(2)

({'age': 73,
  'name': ['Ezekiel', 'Woodard'],
  'occupation': 'Yard Manager',
  'telephone': '+1-331-365-4996',
  'address': {'address': '1177 Arballo Gardens', 'city': 'Passaic'},
  'credit-card': {'number': '3439 860587 17055', 'expiration-date': '06/21'}},
 {'age': 2,
  'name': ['Maire', 'Sharp'],
  'occupation': 'Pipe Insulator',
  'telephone': '+15862379883',
  'address': {'address': '1151 Rausch Avenue', 'city': 'Jeffersontown'},
  'credit-card': {'number': '3794 644559 58638', 'expiration-date': '08/22'}})

In [6]:
b.filter(lambda record: record['age'] > 30).take(2)
# Select only people over 30

({'age': 73,
  'name': ['Ezekiel', 'Woodard'],
  'occupation': 'Yard Manager',
  'telephone': '+1-331-365-4996',
  'address': {'address': '1177 Arballo Gardens', 'city': 'Passaic'},
  'credit-card': {'number': '3439 860587 17055', 'expiration-date': '06/21'}},
 {'age': 75,
  'name': ['Tiffaney', 'Potts'],
  'occupation': 'Interior Designer',
  'telephone': '+1-928-475-0361',
  'address': {'address': '989 Acorn Manor', 'city': 'Holladay'},
  'credit-card': {'number': '5404 2959 7288 5096',
   'expiration-date': '12/22'}})

In [7]:
b.map(lambda record: record['occupation']).take(2)
# Select the occupation field

('Yard Manager', 'Pipe Insulator')

In [8]:
result = (b.filter(lambda record: record['age'] > 30)
.map(lambda record: record['occupation'])
.frequencies(sort= True)
.topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

In [9]:
result.compute()

[('Forensic Scientist', 52),
 ('Judge', 52),
 ('Training Manager', 52),
 ('Bookmaker', 48),
 ('Homeworker', 48),
 ('Licensee', 46),
 ('Town Clerk', 46),
 ('Hot Foil Printer', 46),
 ('Picture Editor', 46),
 ('Lighthouse Keeper', 46)]

In [10]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.00.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.01.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.02.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.03.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.04.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.05.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.06.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.07.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.08.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.09.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.10.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/processed.11.json',
 'C:/Users/test.DESKTOP-ODO5P7A/Desktop/DASK Example/data/proces

In [11]:
b.take(1)

({'age': 73,
  'name': ['Ezekiel', 'Woodard'],
  'occupation': 'Yard Manager',
  'telephone': '+1-331-365-4996',
  'address': {'address': '1177 Arballo Gardens', 'city': 'Passaic'},
  'credit-card': {'number': '3439 860587 17055', 'expiration-date': '06/21'}},)

In [12]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']
    }

b.map(flatten).take(1)

({'age': 73,
  'occupation': 'Yard Manager',
  'telephone': '+1-331-365-4996',
  'credit-card-number': '3439 860587 17055',
  'credit-card-expiration': '06/21',
  'name': 'Ezekiel Woodard',
  'street-address': '1177 Arballo Gardens',
  'city': 'Passaic'},)

In [13]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,73,Yard Manager,+1-331-365-4996,3439 860587 17055,06/21,Ezekiel Woodard,1177 Arballo Gardens,Passaic
1,2,Pipe Insulator,+15862379883,3794 644559 58638,08/22,Maire Sharp,1151 Rausch Avenue,Jeffersontown
2,25,Recovery Vehicle Coordinator,+15802338970,5247 5511 6831 4759,10/16,Joette Joseph,1387 Swiss Stravenue,Carpentersville
3,75,Interior Designer,+1-928-475-0361,5404 2959 7288 5096,12/22,Tiffaney Potts,989 Acorn Manor,Holladay
4,71,Horse Dealer,+16097756691,3439 265888 08784,05/18,Susann Sheppard,688 Morningside Lake,Lodi


In [14]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

2024-09-10 14:07:24,684 - distributed.scheduler - ERROR - Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments



RuntimeError: Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments
